In [1]:
import os; os.chdir("/home/ubuntu/coma/")
import tensorflow as tf
import numpy as np
import ipywidgets as widgets
import yaml
from copy import deepcopy
import json
import re
import itertools
import numpy as np
import scipy

from scipy.spatial import procrustes as procrustes

import glob, os, json
import pandas as pd

from main import generate_model #, run as train
from VTK.VTKMesh import VTKObject as Mesh
from cardiac_mesh import CardiacMesh
from sklearn.decomposition import PCA, IncrementalPCA as IPCA

## Definitions of paths and constants

In [2]:
partitions = ["LV", "RV"]

data_files = {
    "LV": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_all_subjects/train.npy",
    "LV_endo": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_endo_all_subjects/LVED_all_subjects.npy",
    "RV": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/RV/RV_all.npy"
}

ids_files = {
    "LV": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_all_subjects/LVED_all_subjects_subj_ids.txt",
    "LV_endo": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_endo_all_subjects/LVED_all_subjects_subj_ids.txt",
    "RV": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/RV/RV_all_subject_ids.txt"
}

# meshes_dir = "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/"
# output_dir = "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/"
# checkpoints_dir = "{}/checkpoints".format(output_dir)

# subparts_ids = CardiacMesh.subparts_ids

In [3]:
all_meshes = np.load(data_files["LV"])

In [4]:
partitions = ["LV", "RV"]
reference_mesh_file = "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/vtk_meshes/1000336/output.001.vtk"

In [5]:
def _assign_set(data):
    training_ids = [(x, "training") for x in data.train_ids]
    validation_ids = [(x, "validation") for x in data.val_ids]
    testing_ids = [(x, "testing") for x in data.test_ids]
    return dict(training_ids + testing_ids + validation_ids)

In [13]:
perf_dfs = []

for partition in partitions:
    
  cardiac_data = CardiacMesh(
   nTraining = 1600,
   nVal = 200,
   train_file = data_files[partition],
   ids_file = ids_files[partition],
   reference_mesh_file = reference_mesh_file,
   pca_n_comp = [8],
   subpart = CardiacMesh.subparts_ids[partition]
  )  

  meshes_1d_train = np.array([x.flatten() for x in cardiac_data.vertices_train])  
  meshes_1d_val   = np.array([x.flatten() for x in cardiac_data.vertices_val])
  meshes_1d_test  = np.array([x.flatten() for x in cardiac_data.vertices_test])
  
  meshes_1d  = np.concatenate((meshes_1d_train, meshes_1d_val, meshes_1d_test))

  all_ids = cardiac_data.train_ids + cardiac_data.val_ids + cardiac_data.test_ids    

  partition_ids = _assign_set(cardiac_data)

  for n in range(1, 33):
    
    print(n)
    pp = PCA(n).fit(meshes_1d_train)
    meshes_reduced = pp.transform(meshes_1d)
    meshes_reconstructed = pp.inverse_transform(meshes_reduced)
  
    mse = ((meshes_1d - meshes_reconstructed)**2).mean(axis=1)
     
    shuffled_ids = range(meshes_reconstructed.shape[0])
    np.random.shuffle(shuffled_ids)        
    mse_shuffled = ((meshes_1d - meshes_reconstructed[shuffled_ids,:])**2).mean(axis=1)
    
    # if n == 1: from IPython import embed; embed()
    perf_df = pd.DataFrame({
      "model_id": ["PCA__{}__{}_comps".format(partition, str(n))] * len(all_ids),
      "subject_id": all_ids, 
      "dataset": [partition_ids[x] for x in all_ids],
      "partition": [partition] * len(all_ids),
      "mse": mse,
      "mse_shuffled": mse_shuffled,
      "n": [n] * len(all_ids),      
    })     

    perf_dfs.append(perf_df)

Vertices normalized
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
Vertices normalized
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [15]:
perf_df_all = pd.concat(perf_dfs)
perf_df_all


perf_df_all.to_csv("/MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/performance_summary/PCA.csv", index_label="run_id")